In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df=pd.read_excel("/Users/jass/Desktop/Mant_UPL/LBT/Atrasos.xlsx")

In [3]:
df['Cod. Atraso'] = df['Cod. Atraso'].astype('string')
df = df.loc[~(df['Cod. Atraso'].isin(['Cod. Atraso']))]
#df['min'] = pd.to_numeric(df['min'], downcast='integer', errors='coerce')
df['Min.'] = df['Min.'].astype('Int64')

In [4]:
cleaned_column_names = (df.columns
                        .str.strip()
                        .str.replace('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))', r'_\1')
                        .str.lower()
                        .str.replace('[ _-]+', '_')
                        .str.replace('[}{)(><.!?\\\\:;,-]', ''))
df.columns = cleaned_column_names
df = df.loc[~(df['cod_atraso'].isna())]
df['fecha_inicio'] = pd.to_datetime(df['fecha_inicio'], infer_datetime_format=True)
df['cod_atraso'] = df['cod_atraso'].astype('string')
df['ubicación_técnica'] = df['ubicación_técnica'].astype('string')

/var/folders/7y/6r4vh83n39q935n6pmf1h7wh0000gn/T/ipykernel_4181/1772978148.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))', r'_\1')
/var/folders/7y/6r4vh83n39q935n6pmf1h7wh0000gn/T/ipykernel_4181/1772978148.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('[ _-]+', '_')
/var/folders/7y/6r4vh83n39q935n6pmf1h7wh0000gn/T/ipykernel_4181/1772978148.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('[}{)(><.!?\\\\:;,-]', ''))


In [5]:
df_2020 = df.loc[df["fecha_inicio"] >= '2016-01-01 00:00:00']
df_2020['Año'] = df_2020['fecha_inicio'].dt.year 
df_2020['Mes'] = df_2020['fecha_inicio'].dt.month 
df_2020 = df_2020.loc[df_2020['cod_atraso'].isin(['AE', 'AM'])] #,"AO"
df_2020 = df_2020.set_index('fecha_inicio').resample('1W').sum().reset_index()
df_2020["MMA_4Week"] = df_2020['min'].rolling(window=4).mean()   
#df_2020["MMA_60"] = df_2020['min'].rolling(window=30).mean()   
#df_2020

In [6]:


fig = px.line(df_2020.sort_values(by=['fecha_inicio'], ascending=[True]).dropna(subset=['MMA_4Week']), x='fecha_inicio', y='MMA_4Week', 
              template='plotly_white',line_shape='spline', range_y=(0, 1500))
fig.update_layout(xaxis_rangeselector_buttons=list([
                dict(label="1m", count=1, step="month", stepmode="backward"),
                dict(label="6m", count=6, step="month", stepmode="backward"),
                dict(label="YTD", count=1, step="year", stepmode="todate"),
                dict(label="1y", count=1, step="year", stepmode="backward"),
                dict(step="all")
            ]))




fig.update_xaxes(title_text='')
fig.update_yaxes(title_text='Minutos')
#fig.update_traces(textposition="outside")
fig.add_hline(y=480,line_width=3, line_dash="dash", line_color="red",annotation_text="Objetivo Semanal Máximo 8 Horas", annotation_position="bottom right")
fig.update_layout(title_text='Performance Mantenimiento LBT 2022 ( Media Móvil 4 Semanas)', title_x=0.5)
fig.write_html("/Users/jass/Desktop/Mant_UPL/LBT/LBT_4week1.html")
fig